**ADK Notebook for LLM Auditor**

https://github.com/google/adk-samples/tree/main/python/agents/llm-auditor

In [ ]:
pip install poetry


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.6/281.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.6/338.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 87.9 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/google/adk-samples.git

In [ ]:
import os

os.chdir("/content/adk-samples/python/agents/llm-auditor")
print("Now in:", os.getcwd())

!pwd
!ls

!poetry install

Now in: /content/adk-samples/python/agents/llm-auditor
/content/adk-samples/python/agents/llm-auditor
deployment  llm_auditor			  pyproject.toml  tests
eval	    llm_auditor_architecture.png  README.md
Creating virtualenv llm-auditor-zsBukavj-py3.11 in /root/.cache/pypoetry/virtualenvs
Updating dependencies
Resolving dependencies... (15.9s)

Package operations: 103 installs, 0 updates, 0 removals

  - Installing zipp (3.23.0): Pending...
  - Installing zipp (3.23.0): Downloading... 0%
  - Installing zipp (3.23.0): Downloading... 100%
  - Installing zipp (3.23.0): Installing...
  - Installing zipp (3.23.0)
  - Installing importlib-metadata (8.7.0): Pending...
  - Installing protobuf (6.32.1): Pending...
  - Installing pyasn1 (0.6.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing protobuf (6.32.1): Pending...
  - Installing pyasn1 (0.6.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing importlib-metadata (8.7.0): Downloadi

In [ ]:
import os

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "true"
os.environ["GOOGLE_CLOUD_PROJECT"] = "instr-cs795-fall25-hqin-1"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"
os.environ["GOOGLE_CLOUD_STORAGE_BUCKET"] = "instr-cs795-fall25-hqin-1-tstilwel"


In [ ]:
import google.auth
creds, project = google.auth.default()
print("Project:", project)


Project: instr-cs795-fall25-hqin-1


In [ ]:
!pip install dotenv
!poetry add google-adk google-genai python-dotenv


The following packages are already present in the pyproject.toml and will be skipped:

  - google-adk
  - google-genai
  - python-dotenv

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [ ]:
import asyncio, nest_asyncio
from google.adk.runners import InMemoryRunner
from google.genai.types import Part, UserContent
from llm_auditor.agent import root_agent

user_input = "Double check this: Earth is further away from the Sun than Mars."
runner = InMemoryRunner(agent=root_agent)

async def main():
    # async: create a session
    session = await runner.session_service.create_session(
        app_name=runner.app_name, user_id="test_user"
    )

    content = UserContent(parts=[Part(text=user_input)])

    # NOTE: run(...) is a sync generator / iterable — no await, no async for
    for event in runner.run(
        user_id=session.user_id,
        session_id=session.id,
        new_message=content,
    ):
        if getattr(event, "content", None):
            for part in getattr(event.content, "parts", []):
                if getattr(part, "text", None):
                    print(part.text)

# Make it work in Colab whether or not a loop already exists
try:
    asyncio.get_running_loop()
    nest_asyncio.apply()
    await main()
except RuntimeError:
    asyncio.run(main())


## Fact-Checking Report: Earth's Distance from the Sun vs. Mars

### Overall Assessment

### OVERALL VERDICT: Inaccurate

### OVERALL JUSTIFICATION:
The answer provided claims that "Earth is further away from the Sun than Mars." However, extensive research from multiple reliable sources consistently shows that Mars is, on average, significantly further from the Sun than Earth. Earth's average distance from the Sun is approximately 93 million miles (150 million kilometers), while Mars' average distance is about 142 million miles (228 million kilometers). Therefore, the original statement is factually incorrect.

### Claim-by-Claim Verification:

*   **CLAIM:** Earth is further away from the Sun than Mars.
    *   **Part in the answer text:** "Earth is further away from the Sun than Mars."
    *   **VERDICT:** Inaccurate
    *   **JUSTIFICATION:** Reliable sources indicate that Earth's average distance from the Sun is approximately 93 million miles (150 million kilometers), which is defi